In [8]:
# Enable Pin 32 / PWM0 from terminal 
!busybox devmem 0x700031fc 32 0x45
!busybox devmem 0x6000d504 32 0x2
# Enable Pin 33 / PWM2
!busybox devmem 0x70003248 32 0x46
!busybox devmem 0x6000d100 32 0x00

In [1]:
import ipywidgets.widgets as widgets
import Jetson.GPIO as GPIO
import traitlets
from traitlets.config.configurable import Configurable
from traitlets.config.configurable import SingletonConfigurable
import time
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BOARD)

In [2]:
class DCMotor(Configurable):
    value = traitlets.Float()

    def __init__(self, pwm_pin, ctrl_pin1, ctrl_pin2, **kwargs):
        self.pwm_pin = pwm_pin
        self.ctrl_pin1 = ctrl_pin1
        self.ctrl_pin2 = ctrl_pin2
        # Motor calibration
        self.alpha = kwargs.get('alpha', 1.0)
        self.beta = kwargs.get('beta', 0.0)
        # GPIO setup
        GPIO.setup(self.pwm_pin, GPIO.OUT)
        self.pwm = GPIO.PWM(self.pwm_pin, 50)  # (channel, frequency)
        self.pwm.start(0)
        self.speed = 0
        GPIO.setup(self.ctrl_pin1, GPIO.OUT, initial=GPIO.LOW)
        GPIO.setup(self.ctrl_pin2, GPIO.OUT, initial=GPIO.LOW)

    @traitlets.observe('value')
    def _observe_value(self, change):
        self._write_value(change['new'])

    def _write_value(self, value):
        """Sets motor value between [-1, 1]"""
        mapped_value = int(100.0 * (self.alpha * value + self.beta))
        speed = min(max(abs(mapped_value), 0), 100)
        self.set_speed(speed)
        if mapped_value > 0:
            self.rotate_forward()
        else:
            self.rotate_backward()

    def set_speed(self, normalized_speed):
        # normalized_speed in percentange from 0 - 100
        self.pwm.ChangeDutyCycle(normalized_speed)
        self.speed = normalized_speed

    def rotate_forward(self):
        GPIO.output(self.ctrl_pin1, GPIO.LOW)
        GPIO.output(self.ctrl_pin2, GPIO.HIGH)

    def rotate_backward(self):
        GPIO.output(self.ctrl_pin1, GPIO.HIGH)
        GPIO.output(self.ctrl_pin2, GPIO.LOW)

    def stop(self):
        GPIO.output(self.ctrl_pin1, GPIO.LOW)
        GPIO.output(self.ctrl_pin2, GPIO.LOW)

class JDuck(SingletonConfigurable):
    def __init__(self, *args, **kwargs):
        self.left_motor = DCMotor(32, 36, 38, alpha=1.0)
        self.right_motor = DCMotor(33, 35, 37, alpha=1.0)
        self.left_motor.set_speed(50)
        self.right_motor.set_speed(50)

    def set_speeds(self, left_speed, right_speed):
        self.left_motor.set_speed(left_speed)
        self.right_motor.set_speed(right_speed)

    def move_forward(self):
        self.left_motor.rotate_forward()
        self.right_motor.rotate_forward()

    def move_backward(self):
        self.left_motor.rotate_backward()
        self.right_motor.rotate_backward()

    def turn_left(self):
        self.left_motor.rotate_backward()
        self.right_motor.rotate_forward()

    def turn_right(self):
        self.left_motor.rotate_forward()
        self.right_motor.rotate_backward()

    def stop(self):
        self.left_motor.stop()
        self.right_motor.stop()

In [4]:
controller = widgets.Controller(index=0)  # replace with index of your controller
display(controller)

Controller()

In [10]:
jduck = JDuck()

In [11]:
left_link = traitlets.dlink((controller.axes[1], 'value'), (jduck.left_motor, 'value'))
right_link = traitlets.dlink((controller.axes[3], 'value'), (jduck.right_motor, 'value'))